# Test McpHub Module

This notebook contains tests for the McpHub class implementation in `src/autocoder/common/mcp_hub.py`.

In [1]:
import os
import sys
import json
import asyncio
import tempfile
from pathlib import Path

from autocoder.common.mcp_hub import McpHub, McpServer, McpTool, McpResource, McpResourceTemplate

In [2]:
# Helper function for async tests
async def run_test(test_func):
    """Helper to run async test functions"""
    try:
        await test_func()
        print("✅ Test passed")
    except AssertionError as e:
        print(f"❌ Test failed: {str(e)}")
    except Exception as e:
        print(f"❌ Test error: {str(e)}")

## Test McpHub Initialization

In [3]:
async def test_init():
    # Create a temporary settings file
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        json.dump({"mcpServers": {}}, f)
        settings_path = f.name
    
    try:
        # Test initialization
        hub = McpHub(settings_path)
        assert not hub.is_connecting, "Hub should not be connecting initially"
        assert hub.connections == {}, "Connections should be empty initially"
        
        # Test settings file creation
        assert Path(settings_path).exists(), "Settings file should exist"
        with open(settings_path) as f:
            settings = json.load(f)
            assert "mcpServers" in settings, "Settings should have mcpServers key"
            
    finally:
        # Cleanup
        os.unlink(settings_path)

await run_test(test_init)

✅ Test passed


## Test Server Model Classes

In [ ]:
def test_models():
    # Test McpTool
    tool = McpTool(
        name="test-tool",
        description="A test tool",
        input_schema={"type": "object"}
    )
    assert tool.name == "test-tool"
    assert tool.description == "A test tool"
    
    # Test McpResource
    resource = McpResource(
        uri="test://resource",
        name="test-resource",
        description="A test resource",
        mime_type="text/plain"
    )
    assert resource.uri == "test://resource"
    assert resource.mime_type == "text/plain"
    
    # Test McpResourceTemplate
    template = McpResourceTemplate(
        uri_template="test://{param}",
        name="test-template",
        description="A test template",
        mime_type="text/plain"
    )
    assert template.uri_template == "test://{param}"
    
    # Test McpServer
    server = McpServer(
        name="test-server",
        config="{}",
        status="disconnected"
    )
    assert server.status == "disconnected"
    assert len(server.tools) == 0
    
    print("✅ All model tests passed")

test_models()

## Test Server Connection Management

In [ ]:
async def test_server_management():
    # Create a temporary settings file
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        settings = {
            "mcpServers": {
                "test-server": {
                    "command": "echo",
                    "args": ["test"]
                }
            }
        }
        json.dump(settings, f)
        settings_path = f.name
    
    try:
        hub = McpHub(settings_path)
        
        # Test get_servers() with empty connections
        assert len(hub.get_servers()) == 0, "Should have no servers initially"
        
        # Test update_server_connections
        new_config = {
            "test-server-2": {
                "command": "echo",
                "args": ["test2"]
            }
        }
        
        try:
            await hub.update_server_connections(new_config)
        except Exception as e:
            # We expect connection attempts to fail since we're using dummy commands
            pass
        
        # Test shutdown
        await hub.shutdown()
        assert len(hub.connections) == 0, "All connections should be closed after shutdown"
        
    finally:
        # Cleanup
        os.unlink(settings_path)

await run_test(test_server_management)

## Test Tool and Resource Operations

In [ ]:
async def test_operations():
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        settings = {"mcpServers": {}}
        json.dump(settings, f)
        settings_path = f.name
    
    try:
        hub = McpHub(settings_path)
        
        # Test tool call (should fail with no connection)
        try:
            await hub.call_tool("non-existent-server", "test-tool")
            assert False, "Should raise error for non-existent server"
        except ValueError:
            pass
        
        # Test resource read (should fail with no connection)
        try:
            await hub.read_resource("non-existent-server", "test://resource")
            assert False, "Should raise error for non-existent server"
        except ValueError:
            pass
            
    finally:
        await hub.shutdown()
        os.unlink(settings_path)

await run_test(test_operations)

## Integration Test with Mock Server

Note: This requires implementing a mock MCP server for proper testing. Below is a placeholder for the structure.

In [ ]:
class MockMcpServer:
    """A mock MCP server for testing"""
    def __init__(self):
        self.tools = [
            McpTool(name="test-tool", description="A test tool", input_schema={})
        ]
        self.resources = [
            McpResource(uri="test://resource", name="test-resource")
        ]

# TODO: Implement proper mock server integration test
print("ℹ️ Mock server integration test structure prepared")